## Imputación de nulos en variables categóricas

Vamos a describir como imputaremos los valores nulos para las columnas categóricas que tienen nulos, para eso trabajaremos sobre el 'value_counts() para cada una de las columnas:


In [1]:
# importamos las librerías que necesitamos

# Tratamiento de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Imputación de nulos usando métodos avanzados estadísticos
# -----------------------------------------------------------------------
from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# Librerías de visualización
# -----------------------------------------------------------------------
import seaborn as sns
import matplotlib.pyplot as plt
# Configuración
# -----------------------------------------------------------------------
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames

In [2]:
# cargamos el dataframe creado en la lección anterior
df = pd.read_csv("bank-additional_clean.csv", index_col = 0)
df.head(1)

,income,kidhome,teenhome,dt_customer,numwebvisitsmonth,id,age,job,marital,education,default,housing,loan,contact,duration,campaign,pdays,previous,poutcome,empvarrate,conspriceidx,consconfidx,euribor3m,nremployed,y,date,latitude,longitude,contact_month,contact_year,age_cat,hijos_totales
0,161770.0,1.0,0.0,2012-04-04,29.0,089b39d8-e4d0-461b-87d4-814d71e0e079,NaN,housemaid,married,basic 4y,No,No,No,telephone,261,1,NaN,0,nonexistent,1.1,93.994,-36.4,4.857,5191,no,2-agosto-2019,41.495,-71.233,agosto,2019.0,Adultos mayores,1.0 hijos


In [3]:
# Obtenemos la lista de columnas categóricas que tienen nulos
nulos_esta_cat = df[df.columns[df.isnull().any()]].select_dtypes(include = "O").columns
print("Las columnas categóricas que tienen nulos son : \n ")
print(nulos_esta_cat)

Las columnas categóricas que tienen nulos son : 
 
Index(['job', 'marital', 'education', 'default', 'housing', 'loan', 'date',
       'contact_month'],
      dtype='object')


In [4]:
# sacamos el 'value_counts()' de cada una de las columnas categóricas que tienen nulos para saber como es la distribución de sus categorías
for col in nulos_esta_cat:
    print(f"La distribución de las categorías para la columna {col.upper()}")
    display(df[col].value_counts() / df.shape[0])  # display es una función utilizada para mostrar objetos de manera más legible en Jupyter Notebooks o entornos similares. 
    print("........................")

La distribución de las categorías para la columna JOB


admin.           0.252860
blue-collar      0.224512
technician       0.163395
services         0.096791
management       0.070930
retired          0.041628
entrepreneur     0.035395
self-employed    0.034628
housemaid        0.026116
unemployed       0.024721
student          0.021000
Name: job, dtype: float64

........................
La distribución de las categorías para la columna MARITAL


married     0.604628
single      0.281512
divorced    0.111884
Name: marital, dtype: float64

........................
La distribución de las categorías para la columna EDUCATION


...

........................
La distribución de las categorías para la columna DATE


4-septiembre-2019    0.000953
4-agosto-2017        0.000907
22-junio-2019        0.000884
26-febrero-2019      0.000860
20-julio-2017        0.000860
                       ...   
19-agosto-2015       0.000256
20-noviembre-2015    0.000233
4-abril-2017         0.000233
21-diciembre-2019    0.000233
14-abril-2015        0.000233
Name: date, Length: 1860, dtype: float64

........................
La distribución de las categorías para la columna CONTACT_MONTH


noviembre     0.083791
octubre       0.083698
julio         0.083558
marzo         0.083465
abril         0.083302
febrero       0.083186
septiembre    0.083000
mayo          0.082488
junio         0.082279
agosto        0.082000
enero         0.081791
diciembre     0.081674
Name: contact_month, dtype: float64

........................


Decidamos ahora que hacer con cada una de las columnas: 

1. **job**: Dado que hay dos categorías que dominan, `admin` y `blue-collar` con un (25% y 22%) del total de los datos, no podemos decir que haya una categoría dominante, por lo tanto reemplazaremos los nulos por una categoría nueva, por ejemplo "Unknown". 

2. **marital** : Al contrario que en la columna `job`, en esta columna encontramos que el 60% de los clientes pertencen a la categoría de 'married' por lo que en este caso optaremos por reemplazar los nulos por la categoría dominante o moda. 

3. **education** y **housing**: En este caso, nos pasa lo mismo que en la columna `job`, por lo que optaremos por reemplazar por una categoría nueva, de nuevo, "Unknown". 

4. **default**: En esta columna, observamos que hay una categoría dominante "No" con un 79% de los clientes, por lo que reemplazaremos por la moda. 

5. **loan**: Mismo caso que la columna `marital`, en este caso, tenemos que el 82% de los clientes pertenecen a la categoría de "No", por lo que reemplazaremos por esos valores. 

6. **contact_month**: Mismo caso que las columnas de `job`, `education` y `housing`, por lo que reemplazaremos por "Unknown". 

7. **date**: En esta columna, tampoco observamos que haya una clase mayoritaría por lo que optaremos por reemplazar también por "Unknown". 

Resumiendo:
- Columnas que reemplazaremos por la moda:

    - marital

    - loan 

    - default
    
- Columnas que reemplzaremos por ina categoría nueva ("Uknown"):

    - job

    - education

    - housing

    - contact_month
    
    - date



Recuerda que cada conjunto de datos es único, por lo que es importante tomar decisiones basadas en una comprensión profunda del dominio y del impacto potencial en tus análisis. Siempre es una buena idea evaluar el efecto de la imputación en la distribución y calidad de tus datos antes de continuar con el análisis.


In [5]:
# lo primero que vamos a hacer es crear dos listas, una con los nombres de las columnas que reemplazaremos por la moda y otra para las columnas que reemplazaremos por una categoría nueva
columnas_moda = ["marital", "loan", "default"]
columnas_desconocido = ["job", "education", "housing", "contact_month", "date"]

Empezamos reemplazando por la moda las columnas de `marital`,  `loan` y `default`, para eso usaremos el método `.fillna()` de Pandas. El método `fillna()` en Pandas se utiliza para rellenar los valores nulos (NaN) con un valor específico. 

La sintaxis básica del método `fillna()` es la siguiente:

```python
df[col].fillna(value, method, axis, inplace, limit)
```

- `value`: El valor que se utilizará para reemplazar los valores nulos.

- `method` (opcional): Método para rellenar los valores nulos. Puede ser "ffill" para rellenar hacia adelante con el valor anterior, "bfill" para rellenar hacia atrás con el siguiente valor, o None para reemplazar directamente los valores nulos con el valor especificado en `value`.

- `axis`: El eje a lo largo del cual se realizará el relleno. Puede ser 0 para rellenar a lo largo de las filas o 1 para rellenar a lo largo de las columnas.

- `inplace` (opcional): Si se establece en True, el reemplazo se realizará en el DataFrame original en lugar de devolver un nuevo DataFrame.

- `limit` (opcional): El número máximo de valores nulos que se rellenarán.


In [6]:

# iteramos por la lista creada en el paso anterior:
for columna in columnas_moda:
    
    # calculamos la moda para la columna por la que estamos iterando
    moda = df[columna].mode()[0]    
    
    # utilizando el método fillna reemplazamos los valores nulos por la moda calculada en el paso anterior. 
    df[columna] = df[columna].fillna(moda)

# por último chequeamos si se han eliminado los nulos en las columnas de "marital" y "loan"
print("Después del reemplazo usando 'fillna' quedan los siguientes nulos")

df[columnas_moda].isnull().sum()

Después del reemplazo usando 'fillna' quedan los siguientes nulos


marital    0
loan       0
default    0
dtype: int64

Continuamos reemplanzando los valores nulos de las columnas que nos quedan que, recordemos lo haremos creando una nueva categoría. Para esto también usaremos el método `fillna()`. Seguiremos la misma lógica que en el caso anterior, iterar por la lista de columnas `columnas_desconocido` y aplicar el método `.fillna()` a cada una de las columnas. 

In [7]:
# iteramos por la lista de columnas a las que le vamos a cambiar los nulos por "Uknown"
for columna in columnas_desconocido:
    
    # reemplazamos los nulos por el valor Unknown para cada una de las columnas de la lista
    df[columna] = df[columna].fillna("Unknown")
    
# comprobamos si quedan nulos en las columnas categóricas. 
print("Después del reemplazo usando 'fillna' quedan los siguientes nulos")
df[columnas_desconocido].isnull().sum()

Después del reemplazo usando 'fillna' quedan los siguientes nulos


job              0
education        0
housing          0
contact_month    0
date             0
dtype: int64

Hicimos varias cosas en este apartado, guardemos nuestros datos para usarse mas tarde!

In [8]:
# Guarda el DataFrame como un archivo CSV
df.to_csv("bank-additional_clean_2.csv", index=False)